In [1]:
from OnlyCNN import TRepNet as only_cnn
from OnlyResNet import TRepNet as only_resnet
from OnlyWavenet import TRepNet as only_wavenet
from CNNandResNet import TRepNet as cnn_res
from WavenetandResnet import TRepNet as wave_res
from WavenetandCNN import TRepNet as wave_cnn
from NoNoise import TRepNet as no_noise
from TRepNet import TRepNet

import numpy as np
import pandas as pd
import tensorflow as tf
import os
import warnings
import time

warnings.filterwarnings('ignore') 

from tensorflow import keras
from sklearn.preprocessing import RobustScaler, Normalizer, StandardScaler
from sklearn.model_selection import train_test_split
from datasets import load_data, random_benchmark, list_datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, accuracy_score
from Imputation import remove_and_impute
from Models import SAE, CNN_AE, LSTM_AE, GRU_AE, Bi_LSTM_AE, CNN_Bi_LSTM_AE, Causal_CNN_AE, Wavenet, Attention_Bi_LSTM_AE, Attention_CNN_Bi_LSTM_AE, Attention_Wavenet

np.random.seed(7)
tf.random.set_seed(7)

In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def flatten_ts(train, test):
    new_train, new_test = [], []
    train_lens = []
    
    for _, row in train.iterrows():
        for i in row.index:
            train_lens.append(len(row[i]))

    maxlen = np.ceil(np.average(train_lens)).astype(int)
    
    for _, row in train.iterrows():
        new_list = []
        for i in row.index:
            ts = []
            for j in range(len(row[i])):
                ts.append(row[i][j])
            new_list.append(ts)
        new_train.append(pad_sequences(new_list, maxlen=maxlen, dtype='float32'))
        
    for _, row in test.iterrows():
        new_list = []
        for i in row.index:
            ts = []
            for j in range(len(row[i])):
                ts.append(row[i][j])
            new_list.append(ts)
        new_test.append(pad_sequences(new_list, maxlen=maxlen, dtype='float32'))
            
    train_df = pd.DataFrame(np.array(new_train).reshape(train.shape[0], maxlen * train.columns.shape[0]))
    test_df = pd.DataFrame(np.array(new_test).reshape(test.shape[0], maxlen * train.columns.shape[0]))

    scaler = RobustScaler()
    scaler.fit(train_df)
    return scaler.transform(train_df), scaler.transform(test_df), maxlen * train.columns.shape[0]
#     return np.array(train_df), np.array(test_df), maxlen * train.columns.shape[0]

def rnn_reshape(train, test, n_steps, n_features):
#     train, test = flatten_ts(train, test)
    return train.reshape(train.shape[0], n_steps, n_features), test.reshape(test.shape[0], n_steps, n_features)

In [3]:
import os
import tensorflow as tf

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
es = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
# mc = keras.callbacks.ModelCheckpoint('model.h5', save_best_only=True)

In [5]:
# when tuning start with learning rate->mini_batch_size -> 
# momentum-> #hidden_units -> # learning_rate_decay -> #layers 

from tensorflow.keras.utils import plot_model
from sklearn.model_selection import GridSearchCV

data_name = 'DiatomSizeReduction'
univariate = True
print('Data: ', data_name)
scores = []
for fn in [only_cnn, only_resnet, only_wavenet, cnn_res, wave_res, wave_cnn, no_noise, TRepNet]:
    train_x, train_y, test_x, test_y = load_data(data_name, univariate=univariate)    
    #     n_steps = train_x.iloc[0][0].shape[0]
    n_features = train_x.columns.shape[0]

    X_train, X_test, n_steps = flatten_ts(train_x, test_x)
    X_train, X_test = rnn_reshape(X_train, X_test, n_steps // n_features, n_features)

    encoder, decoder = fn(n_steps // n_features, n_features, activation='elu')
    model = keras.models.Sequential([encoder, decoder])

    model.compile(loss="mae", optimizer=keras.optimizers.Nadam(lr=0.001, clipnorm=1.), metrics=['mae'])
    history = model.fit(X_train, X_train, epochs=500, batch_size=16, validation_data=[X_test, X_test], callbacks=[es], verbose=0, shuffle=False)

    # Codings
    codings_train = encoder.predict(X_train)
    codings_test = encoder.predict(X_test)

    #     # RF
    #     rf_clf.fit(codings_train, train_y)
    #     pred = rf_clf.predict(codings_test)
    #     rf_scores = {'accuracy': accuracy_score(test_y, pred), 'f1': f1_score(test_y, pred, average='weighted')}
    #     print('RF >>', rf_scores)

    # SVM
    svm_clf = SVC(random_state=7, gamma='scale')
#     nb_classes = np.unique(train_y).shape[0]
#     train_size = codings_train.shape[0]
#     if train_size // nb_classes < 5 or train_size < 50:
#         svm_clf.fit(codings_train, train_y)
#     else:
#         grid_search = GridSearchCV(svm_clf, {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, np.inf]}, cv=5, iid=False, n_jobs=-1)
#         if train_size <= 10000:
#             grid_search.fit(codings_train, train_y)
#         else:
#             codings_train, _, train_y, _  = train_test_split(codings_train, train_y, train_size=10000, random_state=7, stratify=train_y)
#             grid_search.fit(codings_train, train_y)       
#         svm_clf = grid_search.best_estimator_

#         svm_clf.fit(codings_train, train_y)

    svm_clf.fit(codings_train, train_y)
    pred = svm_clf.predict(codings_test)

    svm_scores = {'accuracy': accuracy_score(test_y, pred), 'f1': f1_score(test_y, pred, average='weighted')}
    scores.append(svm_scores)
    print('SVM >>', svm_scores)
pd.DataFrame(scores).to_csv('./results/ablation-'+data_name+'.csv', index=False)

Data:  DiatomSizeReduction
SVM >> {'accuracy': 0.8856209150326797, 'f1': 0.842923495438463}
SVM >> {'accuracy': 0.5686274509803921, 'f1': 0.4339709373485465}
SVM >> {'accuracy': 0.9183006535947712, 'f1': 0.9027866068707274}
SVM >> {'accuracy': 0.8758169934640523, 'f1': 0.827534194051909}
SVM >> {'accuracy': 0.8888888888888888, 'f1': 0.8403307905971613}
SVM >> {'accuracy': 0.9084967320261438, 'f1': 0.8868189952676339}
SVM >> {'accuracy': 0.8725490196078431, 'f1': 0.8243738086701949}
SVM >> {'accuracy': 0.8888888888888888, 'f1': 0.8410942714392521}
